In [2]:
# training 
import warnings
warnings.filterwarnings("ignore")
import os, time
from cvtk.core import Context, ObjectDetectionDataset, TFFasterRCNN
from cvtk.utils import detection_utils

# Disable printing of logging messages
from azuremltkbase.logging import ToolkitLogger
ToolkitLogger.getInstance().setEnabled(False)

# Initialize the context object
out_root_path = "../../../cvtk_output_400"
Context.create(outputs_path=out_root_path, persistent_path=out_root_path, temp_path=out_root_path)

from matplotlib import pyplot as plt
# Display the images
%matplotlib inline
import xml.etree.ElementTree

In [9]:
# initialize variables
image_folder = r"../sample_data/"
annotations_dir = "Annotation_400"
image_subdirectory = 'Data-400'

# training parameters
score_threshold = 0.0       # Threshold on the detection score, use to discard lower-confidence detections.
max_total_detections = 300  # Maximum number of detections. A high value will slow down training but might increase accuracy.

# to get good results, use a larger value for num_steps, e.g., 5000.
num_steps = 200
learning_rate = 0.001 # learning rate


# save only while training
save_model_path = out_root_path + "/frozen_model/faster_rcnn_400.model" # Please save your model to outside of your AML workbench project folder because of the size limit of AML project


# set deployment name
deployment_name = "wsdeployment"

In [4]:
listdir = os.listdir(image_folder + annotations_dir)
for filename in listdir:
    name = image_folder + annotations_dir + '/' + filename
    
    et = xml.etree.ElementTree.parse(name)
    et.write(name)

In [5]:
# training

data_train = ObjectDetectionDataset.create_from_dir(dataset_name='training_dataset', data_dir=image_folder,
                                                    annotations_dir=annotations_dir, image_subdirectory=image_subdirectory)

# Show some statistics of the training image, and also give one example of the ground truth rectangle annotations
data_train.print_info()
# _ = data_train.images[2].visualize_bounding_boxes(image_size = (10,10))

F1 2018-07-25 10:45:42,183 INFO azureml.vision:machine info {"is_dsvm": true, "os_type": "Windows"} 
F1 2018-07-25 10:45:42,190 INFO azureml.vision:dataset creating dataset for scenario=detection 
Dataset name: training_dataset
Total classes: 10, total images: 400
Label-wise object counts:
	Label basket: 58 objects
	Label dishwasher: 65 objects
	Label headboard: 8 objects
	Label paper towel: 69 objects
	Label person: 69 objects
	Label plate: 49 objects
	Label printer: 67 objects
	Label remote control: 25 objects
	Label stuffed animal: 23 objects
	Label toilet: 68 objects
Bounding box width and height distribution:
	Bounding box widths  0/5/25/50/75/95/100-percentile: 8/22/52/88/153/300/621 pixels
	Bounding box heights 0/5/25/50/75/95/100-percentile: 4/18/50/90/182/290/469 pixels


In [6]:
# training

my_detector = TFFasterRCNN(labels=data_train.labels, 
                           score_threshold=score_threshold, 
                           max_total_detections=max_total_detections)

In [7]:
print(my_detector.class_map)

None


In [10]:
# change steps while training 
print("tensorboard --logdir={}".format(my_detector.train_dir))



start_train = time.time()
my_detector.train(dataset=data_train, num_steps=num_steps, 
                  initial_learning_rate=learning_rate)
end_train = time.time()
print(end_train-start_train)

tensorboard --logdir=C:\Users\admin-dsvm\Documents\cvp_project\cvtk_output_400\temp_faster_rcnn_resnet50\models\train
F1 2018-07-25 10:53:41,187 INFO azureml.vision:Fit starting in experiment 261243702 
F1 2018-07-25 10:53:41,188 INFO azureml.vision:model starting trainging for scenario=detection 
Using existing checkpoint file that's saved at 'C:\Users\admin-dsvm\Documents\cvp_project\cvtk_output_400\models\detection\faster_rcnn_resnet50_coco_2018_01_28\model.ckpt.index'.
TFRecords creation started.
F1 2018-07-25 10:53:41,194 INFO On image 0 of 400
F1 2018-07-25 10:53:41,298 INFO On image 100 of 400
F1 2018-07-25 10:53:41,396 INFO On image 200 of 400
F1 2018-07-25 10:53:41,496 INFO On image 300 of 400
TFRecords creation completed.
Training started.
Training progressing: step 0 ...
Training progressing: step 100 ...
F1 2018-07-25 11:32:03,412 INFO Graph Rewriter optimizations enabled
Converted 275 variables to const ops.
F1 2018-07-25 11:32:10,922 INFO 3159 ops in the final graph.
F1 2

In [11]:
print("tensorboard --logdir={} --port=8008".format(my_detector.eval_dir))

tensorboard --logdir=C:\Users\admin-dsvm\Documents\cvp_project\cvtk_output_400\temp_faster_rcnn_resnet50\models\eval --port=8008


### Step 6: Save the model

In [12]:
my_detector.save(save_model_path) #(r'C:\Users\admin-dsvm\Documents\cvp_project\cvtk_output\frozen_model\faster_rcnn_100.model')

F1 2018-07-25 13:03:40,304 INFO Graph Rewriter optimizations enabled
Converted 275 variables to const ops.
F1 2018-07-25 13:03:46,152 INFO 3159 ops in the final graph.


In [52]:
detections_dict = my_detector.score(r'C:\Users\admin-dsvm\Documents\cvp_project\classification\detection\sample_data\Data-100\1.jpg')


In [53]:
detections_dict['detection_scores']

array([0.04102655, 0.03830199, 0.02843399, 0.02680402, 0.02664556,
       0.02293397, 0.02276698, 0.02069485, 0.01996751, 0.01966314,
       0.01820783, 0.01810025, 0.0178801 , 0.01693222, 0.01683766,
       0.01677918, 0.01664479, 0.01635437, 0.01503515, 0.01488737,
       0.01457681, 0.01411349, 0.01401917, 0.01197707, 0.0118463 ,
       0.01169397, 0.01147731, 0.01146626, 0.01093624, 0.01065345,
       0.01060342, 0.01045635, 0.0100084 , 0.00913158, 0.0090277 ,
       0.00900642, 0.00894993, 0.00888191, 0.00886698, 0.00884378,
       0.00863337, 0.00852825, 0.00839657, 0.00823467, 0.00816436,
       0.00802594, 0.00801599, 0.00756569, 0.00752374, 0.00746532,
       0.00746292, 0.00734097, 0.00731743, 0.00709262, 0.0069861 ,
       0.00670625, 0.00669661, 0.00654807, 0.00652158, 0.0064776 ,
       0.00639548, 0.00635433, 0.00632431, 0.00617288, 0.00616848,
       0.00594472, 0.00590741, 0.00581393, 0.00567456, 0.00565433,
       0.00562199, 0.00557957, 0.00551915, 0.00548276, 0.00544

In [50]:
look_up = dict((v,k) for k,v in my_detector.class_map.items())

In [54]:
pred_labels = []
for i in range(detections_dict['num_detections']):
    if detections_dict['detection_scores'][i] > 0.3:

        pred_labels.append(look_up[detections_dict['detection_classes'][i]])

In [55]:
pred_labels

[]

In [ ]:
from cvtk.operationalization import AMLDeployment



# Create deployment object
# It will use the current deployment environment (you can check it with CLI command "az ml env show").
deploy_obj = AMLDeployment(deployment_name=deployment_name, aml_env="cluster", associated_DNNModel=my_detector, replicas=1)

# Alternatively, you can provide azure machine learning deployment cluster name (environment name) and resource group name
# to deploy your model. It will use the provided cluster to deploy. To do that, please uncomment the following lines to create 
# the deployment object.

# azureml_rscgroup = "<resource group>"
# cluster_name = "<cluster name>"
# deploy_obj = AMLDeployment(deployment_name=deployment_name, associated_DNNModel=my_detector,
#                            aml_env="cluster", cluster_name=cluster_name, resource_group=azureml_rscgroup, replicas=1)

# Check if the deployment name exists, if yes remove it first.
if deploy_obj.is_existing_service():
    AMLDeployment.delete_if_service_exist(deployment_name)
    
# create the webservice
print("Deploying to Azure cluster...")
deploy_obj.deploy()
print("Deployment DONE")

In [ ]:
# Score with existing deployment object

# Score local image with file path
print("Score local image with file path")
image_path_or_url = data_train.images[0].storage_path
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json[:50])

# Score image url and remove image resizing
print("Score image url")
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url)
print("serialized_result_in_json:", serialized_result_in_json[:50])
